In [1]:
import importlib
from main import TimeLoopExperimentController
import architectures.architecture_strategy
import architectures.architecture_constants
import architecture_results.derived_metrics_evaluator
import matplot_results_plotter.matplot_results_plotter

from architectures.architecture_constants import Architecture, GPUMemoryScale, RackSize, base_config, resnet_18_layers

import matplotlib.pyplot as plt

importlib.reload(architectures.architecture_strategy)
importlib.reload(architectures.architecture_constants)
importlib.reload(architecture_results.derived_metrics_evaluator)
importlib.reload(matplot_results_plotter.matplot_results_plotter)

<module 'matplot_results_plotter.matplot_results_plotter' from '/home/workspace/matplot_results_plotter/matplot_results_plotter.py'>

In [2]:
# debug currently set to True, set to False to run
controller = TimeLoopExperimentController(False)

controller.run_all()
controller.plot_results()

No json file found
Running layer 1: l1_conv1
[INFO] 2025-04-25 14:30:21,248 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== mac ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 16 (4*4)
    Compute energy        : 3.27 pJ

    STATS
    -----
    Utilized instances      : 1
    Computes (total)        : 7552892928
    Cycles                  : 7552892928
    Energy (total)          : 24735724339.20 pJ
    Area (total)            : 27624.00 um^2

Level 1
-------
=== output_activation_reg ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 1
        Word bits                       : 16
        Block size                      : 1
        Cluster size                    : 1
        Instances                       : 16 (4*4)
        Shared bandwidth                : -
        Read bandwidth                  : -
        Write bandwidth                 : -
        Multiple buffering              : 1.00
        Effective size                  

INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


KeyboardInterrupt: 